In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#see total no of images and rows in train.csv

'''
import os
import pandas as pd

# Path to your training images and CSV file
image_dir = '/kaggle/input/visual-taxonomy/train_images'
csv_file = '/kaggle/input/visual-taxonomy/train.csv'

# Count the number of images
total_images = len(os.listdir(image_dir))

# Load the CSV file and count the number of rows
train_df = pd.read_csv(csv_file)
total_rows = len(train_df)

# Display the results
print(f'Total number of images: {total_images}')
print(f'Total number of rows in train.csv: {total_rows}')
'''

Total number of images: 70379
Total number of rows in train.csv: 70213


In [ ]:
#see what all images are missing

'''import os
import pandas as pd

# Path to your training images and CSV file
image_dir = '/kaggle/input/visual-taxonomy/train_images'
csv_file = '/kaggle/input/visual-taxonomy/train.csv'

# Load the CSV file
train_df = pd.read_csv(csv_file)

# Get a list of image filenames based on the id, ensuring leading zeros
image_filenames = {f"{int(row['id']):06}.jpg" for _, row in train_df.iterrows()}

# Get a list of image filenames from the directory
existing_filenames = set(os.listdir(image_dir))

# Find images in the directory that are not in the CSV
missing_images = existing_filenames - image_filenames

# Output the number of missing images and the list (if desired)
print(f'Number of missing images: {len(missing_images)}')
print('Missing images:', missing_images)
'''

In [ ]:
#delete unwanted directories in output

'''
import shutil

# Define the directories or files you want to delete
unwanted_dir = '/kaggle/working/'  # Example of a folder to delete

# Check if the directory exists, then delete
if os.path.exists(unwanted_dir):
    shutil.rmtree(unwanted_dir)  # Recursively delete the folder and its contents
    print(f"{unwanted_dir} has been deleted.")
else:
    print(f"{unwanted_dir} does not exist.")
    
    '''

# Step 2: Data Preprocessing

In [ ]:
#1.1 Load the Dataset
'''
import pandas as pd
import os
from PIL import Image

# Load the CSV file containing the metadata (attributes)
train_data = pd.read_csv('/kaggle/input/visual-taxonomy/train.csv')

# Display the first few rows of the dataframe
print(train_data.head())
# Print all column names
print(train_data.columns)
'''

### 2.1 Image Preprocessing

In [ ]:
#2.1.1  image loading and organizing into subfolders
'''
import os
import pandas as pd
import shutil
from tqdm import tqdm

# Load the train.csv to understand the category distribution
train_df = pd.read_csv('/kaggle/input/visual-taxonomy/train.csv')

# Define directories
source_dir = '/kaggle/input/visual-taxonomy/train_images/'
destination_dir = '/kaggle/working/organized_train_images/'

# Create new directories based on categories
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Group images by category and organize them into subfolders
for category in train_df['Category'].unique():
    category_folder = os.path.join(destination_dir, category)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)

# Move images into respective subfolders by category
for index, row in tqdm(train_df.iterrows(), total=len(train_df)):
    image_id = f"{row['id']:06d}.jpg"  # Zero-padded image ID
    category = row['Category']
    src_path = os.path.join(source_dir, image_id)
    dest_path = os.path.join(destination_dir, category, image_id)

    # Move the file
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_path)  # You can use shutil.move if you want to remove the original files

print("Images organized into subfolders by category.")
'''

In [ ]:
# 2.1.2 Image Resizing with Aspect Ratio Preservation and Padding (with Subfolder Preservation)
'''
import os
from PIL import Image

# Image size (target size after resizing with padding)
image_size = (512, 512)

# Function to resize image while preserving aspect ratio and adding padding
def resize_with_padding(image, target_size=image_size):
    # Resize the image while maintaining aspect ratio
    img_w, img_h = image.size
    target_w, target_h = target_size
    
    # Calculate the new size to preserve aspect ratio
    ratio = min(target_w / img_w, target_h / img_h)
    new_size = (int(img_w * ratio), int(img_h * ratio))
    
    # Resize the image
    resized_image = image.resize(new_size, Image.LANCZOS)
    
    # Create a new image with the target size and a black background
    new_image = Image.new("RGB", target_size, (0, 0, 0))
    
    # Paste the resized image onto the center of the black canvas
    new_image.paste(resized_image, ((target_w - new_size[0]) // 2, (target_h - new_size[1]) // 2))
    
    return new_image

# Directory paths
organized_dir = '/kaggle/working/organized_train_images/'  # Input directory with category subfolders
output_dir = '/kaggle/working/resized_images/'  # Output directory for resized images
os.makedirs(output_dir, exist_ok=True)

# Resize and save all images while preserving subfolder structure
for subdir, dirs, files in os.walk(organized_dir):
    for file in files:
        if file.endswith(('jpg', 'jpeg', 'png')):
            image_path = os.path.join(subdir, file)

            # Create corresponding subfolder structure in output directory
            relative_subdir = os.path.relpath(subdir, organized_dir)  # Get relative path to preserve category subfolders
            output_subdir = os.path.join(output_dir, relative_subdir)
            os.makedirs(output_subdir, exist_ok=True)  # Ensure subfolders exist
            
            output_path = os.path.join(output_subdir, file)  # Save to the correct subfolder
            
            # Load image, resize with padding, and save
            image = Image.open(image_path)
            resized_image = resize_with_padding(image)
            resized_image.save(output_path, 'JPEG')
            
print("All images resized with padding and saved with folder structure preserved.")
'''

### AUGUMENTATION

High severity (< 1000 samples):

2 images with both basic and advanced augmentations
1 MixUp image
1 CutMix image
1 original image (copied)
Total: 7 images (6 augmented + 1 original)


Medium severity (1000-5000 samples):

3 images with basic augmentations (50% chance of also getting advanced augmentations)
1 original image (copied)
Total: 4 images (3 augmented + 1 original)


Low severity (5000-10000 samples):

1 image with basic augmentations
1 original image (copied)
Total: 2 images (1 augmented + 1 original)


Very high sample count (> 10000):

20% chance of getting 1 image with basic augmentations
1 original image (copied)
Total: 1-2 images (0-1 augmented + 1 original)

In [ ]:
'''
import pandas as pd

# Load train.csv
train_df = pd.read_csv('/kaggle/input/visual-taxonomy/train.csv')

# Define thresholds for augmentation
augmentation_thresholds = {
    "low": {"limit": 1000, "augment_count": 4},
    "medium": {"limit": 5000, "augment_count": 3},
    "high": {"limit": 10000, "augment_count": 1},
    "very_high": {"limit": float('inf'), "augment_count": 0.2}, # Random 20% of the images
}

# Analyze class distribution for each attribute
class_distributions = {}

for i in range(1, 11):
    attr_col = f'attr_{i}'
    class_distributions[attr_col] = train_df[attr_col].value_counts().to_dict()

# Summary of distributions for further augmentation strategy
class_distributions

'''

In [ ]:
'''
import os
import pandas as pd
import numpy as np
from PIL import Image
import albumentations as A
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split

# Define paths
input_dir = '/kaggle/input/augumented-images/resized_images'
output_dir = '/kaggle/working/augmented_images/'
train_csv_path = '/kaggle/input/visual-taxonomy/train.csv'

# Load and analyze the dataset
def analyze_dataset(csv_path):
    df = pd.read_csv(csv_path)
    attribute_stats = {}
    for col in df.columns:
        if col.startswith('attr_'):
            attribute_stats[col] = df[col].value_counts().to_dict()
    return df, attribute_stats

# Define augmentation strategies
def get_basic_augmentation():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.7),
        A.ShiftScaleRotate(p=0.5),
    ])

def get_advanced_augmentation():
    return A.Compose([
        A.OneOf([
            A.GaussNoise(p=0.5),
            A.GaussianBlur(p=0.5),
        ], p=0.5),
        A.OneOf([
            A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=None, p=0.5),  # Set alpha_affine to None
            A.GridDistortion(p=0.5),
            A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=0.5),
        ], p=0.5),
    ])


def mixup(image1, image2, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    mixed_image = (lam * image1 + (1 - lam) * image2).astype(np.uint8)
    return mixed_image, lam

def cutmix(image1, image2):
    h, w = image1.shape[:2]
    center_x, center_y = np.random.randint(w), np.random.randint(h)
    size = min(w, h) // 2
    x1, y1 = max(0, center_x - size // 2), max(0, center_y - size // 2)
    x2, y2 = min(w, center_x + size // 2), min(h, center_y + size // 2)
    
    image_cut = image1.copy()
    image_cut[y1:y2, x1:x2] = image2[y1:y2, x1:x2]
    return image_cut

def augment_image(image, category, attributes, stats):
    augmented_images = []
    base_filename = os.path.splitext(os.path.basename(image))[0]
    
    # Determine augmentation severity based on the rarest attribute, ignoring NaN values
    min_count = float('inf')
    for attr in attributes:
        if attr in stats and pd.notna(attributes[attr]):
            attr_value = attributes[attr]
            if attr_value in stats[attr]:
                min_count = min(min_count, stats[attr][attr_value])
    
    # If all attributes are NaN, use a default severity
    if min_count == float('inf'):
        min_count = 5000  # Choose medium severity as default
    
    basic_aug = get_basic_augmentation()
    advanced_aug = get_advanced_augmentation()
    
    img = np.array(Image.open(image))

    if min_count < 1000:  # High severity
        num_augmentations = 4
        for i in range(num_augmentations):
            augmented = basic_aug(image=img)['image']
            augmented = advanced_aug(image=augmented)['image']
            aug_filename = f"{base_filename}_high_{i}.jpg"
            augmented_images.append((augmented, aug_filename))
        
        # Apply MixUp and CutMix
        if len(os.listdir(os.path.dirname(image))) > 1:
            other_image = np.random.choice([f for f in os.listdir(os.path.dirname(image)) if f != os.path.basename(image)])
            other_img = np.array(Image.open(os.path.join(os.path.dirname(image), other_image)))
            
            mixed_img, _ = mixup(img, other_img)
            augmented_images.append((mixed_img, f"{base_filename}_mixup.jpg"))
            
            cut_img = cutmix(img, other_img)
            augmented_images.append((cut_img, f"{base_filename}_cutmix.jpg"))
    
    elif min_count < 5000:  # Medium severity
        num_augmentations = 3
        for i in range(num_augmentations):
            augmented = basic_aug(image=img)['image']
            if np.random.random() < 0.5:  # 50% chance of advanced aug
                augmented = advanced_aug(image=augmented)['image']
            aug_filename = f"{base_filename}_medium_{i}.jpg"
            augmented_images.append((augmented, aug_filename))
    
    elif min_count < 10000:  # Low severity
        augmented = basic_aug(image=img)['image']
        aug_filename = f"{base_filename}_low_0.jpg"
        augmented_images.append((augmented, aug_filename))
    
    else:  # Very high sample count
        if np.random.random() < 0.2:  # 20% chance for images with >10000 samples
            augmented = basic_aug(image=img)['image']
            aug_filename = f"{base_filename}_verylow_0.jpg"
            augmented_images.append((augmented, aug_filename))

    return augmented_images

def augment_dataset(input_dir, output_dir, df, stats):
    for category in os.listdir(input_dir):
        category_dir = os.path.join(input_dir, category)
        output_category_dir = os.path.join(output_dir, category)
        os.makedirs(output_category_dir, exist_ok=True)

        for image_name in tqdm(os.listdir(category_dir), desc=f"Augmenting {category}"):
            image_path = os.path.join(category_dir, image_name)
            image_id = int(os.path.splitext(image_name)[0])
            
            # Get attributes for this image
            image_data = df[df['id'] == image_id].iloc[0]
            attributes = {col: image_data[col] for col in df.columns if col.startswith('attr_')}

            augmented_images = augment_image(image_path, category, attributes, stats)

            for aug_img, aug_filename in augmented_images:
                output_path = os.path.join(output_category_dir, aug_filename)
                Image.fromarray(aug_img).save(output_path)

            # Copy original image
            shutil.copy(image_path, os.path.join(output_category_dir, image_name))

def update_train_csv(input_csv, output_dir, output_csv):
    df = pd.read_csv(input_csv)
    new_rows = []

    for category in os.listdir(output_dir):
        category_dir = os.path.join(output_dir, category)
        for image_name in os.listdir(category_dir):
            base_name, _ = os.path.splitext(image_name)
            original_id = int(base_name.split('_')[0])
            
            original_row = df[df['id'] == original_id].iloc[0]
            new_row = original_row.copy()
            new_row['id'] = base_name
            new_rows.append(new_row)

    updated_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    updated_df.to_csv(output_csv, index=False)

# Main execution
if __name__ == "__main__":
    print("Starting data augmentation process...")
    
    print("Analyzing dataset...")
    df, stats = analyze_dataset(train_csv_path)
    
    print("Augmenting images...")
    augment_dataset(input_dir, output_dir, df, stats)
    
    print("Updating train CSV...")
    update_train_csv(train_csv_path, output_dir, '/kaggle/working/updated_train.csv')
    
    print("Augmentation completed. Updated train CSV created.")
    print(f"Augmented images are saved in: {output_dir}")
    print("Updated CSV file: /kaggle/working/updated_train.csv")
    
'''

In [ ]:
'''
# Create a subdirectory
os.makedirs('/kaggle/working/my_dataset/', exist_ok=True)

# Move the file to the new directory
shutil.move('/kaggle/working/updated_train.csv', '/kaggle/working/my_dataset/updated_train.csv')

'''

In [ ]:
'''
import os
import pandas as pd

# Path to your training images and CSV file
image_dir = '/kaggle/input/augumented-images/augmented_images'
csv_file = '/kaggle/input/augumented-images/final_preprocessed_images/updated_train_with_preprocessed.csv'

# Count the total number of images in the direct subfolders
total_images = 0
for subfolder in os.listdir(image_dir):
    subfolder_path = os.path.join(image_dir, subfolder)
    if os.path.isdir(subfolder_path):
        total_images += len([f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))])

# Load the CSV file and count the number of rows
train_df = pd.read_csv(csv_file)
total_rows = len(train_df)

# Display the results
print(f'Total number of images: {total_images}')
print(f'Total number of rows in train.csv: {total_rows}')

'''

In [ ]:
# 2. Check how many images are organized in subfolders
'''
for root, dirs, files in os.walk('/kaggle/input/augumented-images/augmented_images'):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        num_files = len(os.listdir(dir_path))
        print(f"Category '{dir_name}' contains {num_files} images.")
'''

In [ ]:
# 3. Visualize some of the augmented images
'''
import matplotlib.pyplot as plt
from PIL import Image
import random

# Get a list of all image paths
all_image_paths = []
for root, dirs, files in os.walk('/kaggle/input/augumented-images/augmented_images'):
    for file in files:
        if file.endswith(('jpg', 'jpeg', 'png')):
            all_image_paths.append(os.path.join(root, file))

# Randomly sample a few images to visualize
sample_images = random.sample(all_image_paths, 5)

# Plotting the images
plt.figure(figsize=(10, 10))
for i, image_path in enumerate(sample_images):
    img = Image.open(image_path)
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

'''


In [ ]:
#Load the new Dataset updated_train.csv
'''
import pandas as pd
import os
from PIL import Image

# Load the CSV file containing the metadata (attributes)
train_data = pd.read_csv('/kaggle/input/augumented-images/my_dataset/updated_train.csv')

# Display the first few rows of the dataframe
print(train_data.head(20))
# Print all column names
print(train_data.columns)

'''

In [ ]:
#Attribute Distribution in new dataset

'''
import seaborn as sns
import matplotlib.pyplot as plt

# List of attributes to check the distribution for
attributes = ['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']  

for attr in attributes:
    # Plot the distribution of each attribute
    plt.figure(figsize=(10, 6))
    sns.countplot(y=train_data[attr], order=train_data[attr].value_counts().index)
    plt.title(f'Distribution of {attr}')
    plt.show()
    
    # Print the distribution of each attribute
    print(f"Distribution of {attr}:")
    attr_distribution = train_data[attr].value_counts()
    for value, count in attr_distribution.items():
        print(f"{value}: {count}")
    print("\n")
    
    '''

### Code for Image Normalization using ImageNet Mean-Std for working well with swim along with image normalization, quality enhancement, deduplication, and optimization

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

# Define paths
INPUT_DIR = '/kaggle/input/augumented-images/augmented_images'
OUTPUT_DIR = '/kaggle/working/Final_images_for_model/'
TRAIN_CSV_PATH = '/kaggle/input/augumented-images/final_preprocessed_images/updated_train_with_preprocessed.csv'
OUTPUT_CSV_FILE = '/kaggle/working/Final_dataset_for_model.csv'  # Changed to .csv file

# Ensure output directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(OUTPUT_CSV_FILE), exist_ok=True)

# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class FashionDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = self._get_image_files()

    def _get_image_files(self):
        image_files = []
        for category in os.listdir(self.image_dir):
            category_path = os.path.join(self.image_dir, category)
            if os.path.isdir(category_path):
                for img_file in os.listdir(category_path):
                    image_files.append((category, img_file))
        return image_files

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        category, img_file = self.image_files[idx]
        img_path = os.path.join(self.image_dir, category, img_file)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        original_id = img_file.split('_')[0].split('.')[0]
        
        try:
            csv_row = self.data[self.data['id'].astype(str) == original_id].iloc[0]
            csv_data = csv_row.to_dict()
            
            return {
                'image': image,
                'img_path': img_path,
                'csv_data': csv_data
            }
        except IndexError:
            print(f"Warning: No matching CSV entry found for image {img_file}")
            return None

def custom_collate(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if len(batch) == 0:
        return None
    return {
        'image': torch.stack([item['image'] for item in batch if isinstance(item, dict) and 'image' in item]),
        'img_path': [item['img_path'] for item in batch if isinstance(item, dict) and 'img_path' in item],
        'csv_data': [item['csv_data'] for item in batch if isinstance(item, dict) and 'csv_data' in item]
    }

def save_processed_images(images, img_paths):
    new_paths = []
    for img, old_path in zip(images, img_paths):
        img = transforms.ToPILImage()(img.cpu())
        
        category = os.path.dirname(old_path).split('/')[-1]
        category_dir = os.path.join(OUTPUT_DIR, category)
        os.makedirs(category_dir, exist_ok=True)
        
        new_filename = os.path.basename(old_path)
        output_path = os.path.join(category_dir, new_filename)
        img.save(output_path)
        new_paths.append(output_path)
    
    return new_paths

BATCH_SIZE = 32
dataset = FashionDataset(csv_file=TRAIN_CSV_PATH, image_dir=INPUT_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, collate_fn=custom_collate)

print("Processing images and preparing new CSV...")
new_data = []
for batch in tqdm(dataloader):
    if batch is None:
        continue
    images, img_paths, csv_data_list = batch['image'], batch['img_path'], batch['csv_data']
    new_paths = save_processed_images(images, img_paths)
    for new_path, csv_data in zip(new_paths, csv_data_list):
        new_row = csv_data.copy()
        new_row['image_path'] = new_path
        new_row['augmentation_type'] = os.path.basename(new_path).split('_')[-1].split('.')[0]
        new_data.append(new_row)

new_df = pd.DataFrame(new_data)
new_df.to_csv(OUTPUT_CSV_FILE, index=False)

print(f"Normalized and enhanced images saved to {OUTPUT_DIR}")
print(f"New CSV file with updated image paths saved to {OUTPUT_CSV_FILE}")

total_processed = sum([len(files) for r, d, files in os.walk(OUTPUT_DIR)])
csv_entries = len(new_df)

print(f"Total processed images: {total_processed}")
print(f"Total CSV entries: {csv_entries}")

if total_processed == csv_entries:
    print("Number of processed images matches CSV entries. Data is ready for model input.")
else:
    print("Warning: Mismatch between processed images and CSV entries. Please check the data.")